INITIAL IMPORT

In [1]:
import torch
import torchvision
import cv2

from imutils import paths
import shutil

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder, Kinetics

from torch.utils.data import DataLoader,Dataset,ConcatDataset
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from matplotlib.image import imread

import time
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

c:\Users\servi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DOWNLOAD THE FACEFORENSIC'S DATASET FROM KAGGLE

In [ ]:
pip install kaggle

In [2]:
os.environ['KAGGLE_USERNAME'] = "stefanoservillo" # username from the json file
os.environ['KAGGLE_KEY'] = "d820211db8d6f4de9f2656d1eb4c2c38" # key to connect with Kaggle API

!kaggle datasets download -d stefanoservillo/deepfake #kaggle command to download the repository

"kaggle" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [ ]:
!unzip deepfake.zip #we have to unzip all the files

DIVIDE DATASET IN TRAIN AND VALIDATION DATASET

In [ ]:
# path to the video repository
DATASET_PATH = "/content/deepfake"

# path for the train and validation set
TRAIN = "train"
VALIDATION = "val"

# split of validation set
VAL_SPLIT = 0.5

In [ ]:
def copy_videos(videoPaths, folder):
	
    # check if the destination folder exists and if not create it
	if not os.path.exists(folder):
		os.makedirs(folder)
	
    # loop over the image paths
	for path in videoPaths:
		
        # grab image name and its label from the path and create
		# a placeholder corresponding to the separate label folder
		videoName = path.split(os.path.sep)[-1]
		label = path.split(os.path.sep)[-2]
		labelFolder = os.path.join(folder, label)
		
        # check to see if the label folder exists and if not create it
		if not os.path.exists(labelFolder):
			os.makedirs(labelFolder)
		
        # construct the destination image path and copy the current
		# image to it
		destination = os.path.join(labelFolder, videoName)
		shutil.copy(path, destination)

In [ ]:
# load all the image paths and randomly shuffle them
print("[INFO] loading video paths...")
videoAlteredPaths = list(paths.list_files(DATASET_PATH+'/altered'))
np.random.shuffle(videoAlteredPaths)

# generate training and validation paths
valAlteredPathsLen = int(len(videoAlteredPaths) * VAL_SPLIT)
trainAlteredPathsLen = len(videoAlteredPaths) - valAlteredPathsLen
trainPaths = videoAlteredPaths[:trainAlteredPathsLen]
valPaths = videoAlteredPaths[trainAlteredPathsLen:]

# copy the training and validation images to their respective
# directories
print("[INFO] copying training and validation videos...")
copy_videos(trainPaths, TRAIN)
copy_videos(valPaths, VALIDATION)

# load all the image paths and randomly shuffle them
print("[INFO] loading video paths...")
videoOriginalPaths = list(paths.list_files(DATASET_PATH+'/original'))
np.random.shuffle(videoOriginalPaths)

# generate training and validation paths
valOriginPathsLen = int(len(videoOriginalPaths) * VAL_SPLIT)
trainOriginPathsLen = len(videoOriginalPaths) - valOriginPathsLen
trainPaths = videoOriginalPaths[:trainOriginPathsLen]
valPaths = videoOriginalPaths[trainOriginPathsLen:]

# copy the training and validation images to their respective
# directories
print("[INFO] copying training and validation videos...")
copy_videos(trainPaths, TRAIN)
copy_videos(valPaths, VALIDATION)

CREATE DIRECTORY TO INSERT THE OPTICAL FLOW IMAGES

In [ ]:
!mkdir /content/valOpticalDataset
!mkdir /content/valOpticalDataset/original
!mkdir /content/valOpticalDataset/altered

In [ ]:
!mkdir /content/trainOpticalDataset
!mkdir /content/trainOpticalDataset/original
!mkdir /content/trainOpticalDataset/altered

DETECT FACE FROM EACH FRAME

In [ ]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)  
    
    for (x, y, w, h) in faces:
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    if (len(faces) == 0):
        return None
    
    return gray[y:y + w, x:x + h]

CALCULATE OPTICAL FLOW AND SAVE THE IMAGE OF THE FRAME IN A DIRECTORY

In [ ]:
def compute_optical_flow(prvs, nxt, number, directory,flag,type1=None):
    
    # Creates an array filled with zero 
    # with the same dimensions of the frame
    rgb_image = cv2.cvtColor(prvs, cv2.COLOR_GRAY2BGR)
    hsv = np.zeros_like(rgb_image)
    hsv[..., 1] = 255

    # Compute the optical flow
    flow = cv2.calcOpticalFlowFarneback(prvs, nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Magnitude and angle of the 2D vectors
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Sets image hue and value according to the optical flow direction
    # and magnitude, then converts HSV to RGB (BGR) color representation
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    if flag:
      plt.imsave('/content/{}OpticalDataset/{}{}.png'.format(type1,directory,number),bgr)
    else:
      plt.imsave('/content/provaOptical/video/{}{}.png'.format(directory,number),bgr)
    
    return bgr

TAKE FRAMES FROM VIDEO, FIND THE FACE AND CALCULATE OPTICAL FLOW ON IT

In [ ]:
def frames(path,type1):
    video = cv2.VideoCapture(path)
    ret, old_frame = video.read()
    if not ret:
        print('No frames grabbed!')
        return
    face1 = detect_face(old_frame)
    path = path.split('/')[1]+'/'+path.split('/')[2][0:-4]

    number = 0
    while(number<20):
        number +=1
        ret, new_frame = video.read()
        if not ret:
            print("The video is finished")
            break
        face2 = detect_face(new_frame)

        try:
          face1 = cv2.resize(face1, (300, 300), interpolation = cv2.INTER_AREA)
          face2 = cv2.resize(face2, (300, 300), interpolation = cv2.INTER_AREA)
          optical_flow = compute_optical_flow(face1, face2, number,path,True,type1)
        except Exception as e:
          print(e)

        old_frame = new_frame
        face1 = face2
    video.release()

videoTrainAlteredPaths = list(paths.list_files('train/altered'))
for i in videoTrainAlteredPaths:
  frames(i,'train')
videoTrainOriginalPaths = list(paths.list_files('train/original'))
for i in videoTrainOriginalPaths:
  frames(i,'train')
videoValAlteredPaths = list(paths.list_files('val/altered'))
for i in videoValAlteredPaths:
  frames(i,'val')
videoValOriginalPaths = list(paths.list_files('val/original'))
for i in videoValOriginalPaths:
  frames(i,'val')

TRANSFORM THE NPARRAY IN TENSOR AND SELECT THE DEVICE

In [ ]:
transfrom = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
BATCH_SIZE=32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GIVE THE CORRECT DIRECTORY TO THE DATASETS

In [ ]:
trainDataset=ImageFolder('/content/trainOpticalDataset',transform=transfrom)
valDataset=ImageFolder('/content/valOpticalDataset',transform=transfrom)

LOAD THE DIFFERENT DATASETS

In [ ]:
train_loader=DataLoader(trainDataset, batch_size=BATCH_SIZE,shuffle=True)
val_loader=DataLoader(valDataset,batch_size=BATCH_SIZE)

SHOW IMAGES

In [ ]:
def showimages(imgs,actual_lbls,pred_lbls=None):
  total = 0
  good = 0
  fig = plt.figure(figsize=(21,12))

  for i,img in enumerate(imgs):
    total +=1
    fig.add_subplot(4,8, i+1)
    y=actual_lbls[i]
    
    if pred_lbls!=None:
      y_pre=pred_lbls[i]
      title="prediction: {0}\nlabel:{1}".format(trainDataset.classes[y_pre],trainDataset.classes[y])
      if trainDataset.classes[y] == trainDataset.classes[y_pre]:
        good +=1
    else: 
      title="Label: {0}".format(trainDataset.classes[y])

    plt.title(title)
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.axis("off")
    plt.imshow(img)
  if pred_lbls != None:
    print("Accuracy: "+ str((good/total)*100)+ '%')
  
plt.show()

inputs, classes = next(iter(train_loader))

showimages(inputs,classes)

TRAIN FUNCTION

In [ ]:
def train(model,loss_fn,dataloader,optimizer,epoch):
  print('\nEpoch : %d'%epoch)
  
  total_loss=0    
  correct=0
  total=0

  model.train()

  for data in tqdm(dataloader):
    
    inputs,labels=data[0].to(device),data[1].to(device)
    
    outputs=model(inputs)
    
    loss=loss_fn(outputs,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
      
  loss=total_loss/len(dataloader)
  accuracy=100.*correct/total
  
  accuracies['train'].append(accuracy)
  losses['train'].append(loss)
  print('Train Loss: %.3f | Accuracy: %.3f'%(loss,accuracy))

TEST FUNCTION

In [ ]:
def test(model,loss_fn,dataloader,epoch):
  model.eval()

  total_loss=0
  correct=0
  total=0

  with torch.no_grad():
    for data in tqdm(dataloader):
      images,labels=data[0].to(device),data[1].to(device)
      
      outputs=model(images)

      loss= loss_fn(outputs,labels)
      total_loss+=loss.item()
      
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()
  
  loss=total_loss/len(dataloader)
  accuracy=100.*correct/total

  losses['val'].append(loss)
  accuracies['val'].append(accuracy)

  print('Test Loss: %.3f | Accuracy: %.3f'%(loss,accuracy)) 

SETUP MODEL

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

loss_fn = nn.CrossEntropyLoss()

params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

TRAIN AND TEST IN EPOCHS

In [ ]:
losses = {'train':[], 'val':[]}
accuracies = {'train':[], 'val':[]}
epochs=20
for epoch in range(1,epochs+1): 
  train(model,loss_fn,train_loader,optimizer_ft,epoch)
  test(model,loss_fn,val_loader,epoch)

GRAPHS

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(accuracies['train'], label='Training Accuracy')
plt.plot(accuracies['val'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(losses['train'], label='Training Loss')
plt.plot(losses['val'], label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

VALIDATION OF THE MODEL

In [ ]:
def predict_images(model,images,actual_label):
  model.eval()
  with torch.no_grad():
    inputs = images.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    showimages(images,actual_label,preds.cpu())

images, classes = next(iter(val_loader))

predict_images(model,images,classes)

TEST A RANDOM VIDEO

In [ ]:
!kaggle datasets download -d stefanoservillo/testtt

In [ ]:
!unzip testtt.zip

In [ ]:
!mkdir /content/provaOptical
!mkdir /content/provaOptical/video

In [ ]:
def frames(path):
    video = cv2.VideoCapture(path)
    ret, old_frame = video.read()
    if not ret:
        print('No frames grabbed!')
        return
    face1 = detect_face(old_frame)
    path = path.split('/')[3][0:-4]

    number = 0
    while(video.isOpened()):
        number +=1
        ret, new_frame = video.read()
        if not ret:
            print("The video is finished")
            break
        face2 = detect_face(new_frame)

        try:
          face1 = cv2.resize(face1, (300, 300), interpolation = cv2.INTER_AREA)
          face2 = cv2.resize(face2, (300, 300), interpolation = cv2.INTER_AREA)
          optical_flow = compute_optical_flow(face1, face2, number,path,False)
        except Exception as e:
          print(e)

        old_frame = new_frame
        face1 = face2
    video.release()

In [ ]:
frames('/content/prova/004.mp4')

In [ ]:
def showimages(imgs,pred_lbls):
  total = 0
  fake = 0
  fig = plt.figure(figsize=(21,12))

  for i,img in enumerate(imgs):
    total +=1
    fig.add_subplot(4,8, i+1)
    
    y_pre=pred_lbls[i]
    title="prediction: {0}".format(trainDataset.classes[y_pre])
    if trainDataset.classes[y_pre] == 'altered':
      fake +=1

    plt.title(title)
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.axis("off")
    plt.imshow(img)
  if pred_lbls != None:
    print("Video fake al "+ str((fake/total)*100)+ '%')

In [ ]:
Dataset=ImageFolder('/content/provaOptical',transform=transfrom)

In [ ]:
data_loader=DataLoader(Dataset,batch_size=BATCH_SIZE)

In [ ]:
def predict_images(model,images):
  model.eval()
  with torch.no_grad():
    inputs = images.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    showimages(images,preds.cpu())

images, classes = next(iter(data_loader))

predict_images(model,images)